# Read Replay Ocean (MOM6) Data, Store to Zarr

Example using UFS2ARCO.MOM6Dataset to read data from a single cycle of the replay dataset.

In [1]:
from os.path import join

import xarray as xr
from datetime import datetime

In [2]:
import sys
sys.path.append("../src")
from UFS2ARCO import MOM6Dataset

## Setup path to read from

In this case, read replay data from the s3 bucket.
Right now, `FV3Dataset` requires a `path_in` callable with 3 entries, so that it can build file paths for a given date (denoting a DA cycle), forecast hours to grab, and file prefixes.

In [3]:
def ocean_path(date: datetime, forecast_hours: list, file_prefixes: list):

    from datetime import timedelta
    upper = "s3://noaa-ufs-gefsv13replay-pds/1deg"
    this_dir = f"{date.year:04d}/{date.month:02d}/{date.year:04d}{date.month:02d}{date.day:02d}{date.hour:02d}"
    files = []
    for fp in file_prefixes:
        for fhr in forecast_hours:
            this_date = date + timedelta(hours=fhr)
            files.append(
                    f"{fp}{this_date.year:04d}_{this_date.month:02d}_{this_date.day:02d}_{this_date.hour:02d}.nc")
    return [join(upper, this_dir, this_file) for this_file in files]

For instance to grab the `ocn_` files at forecast hour `fhr00` 
from the DA cycle at 00:00 Jan 1, 1994, this would be used as follows:

In [4]:
cycle = datetime(1994,1,1,0)
ocean_path(cycle, [0], ["ocn_"])

['s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/ocn_1994_01_01_00.nc']

However, reading from s3 is super slow with `xarray.open_dataset`.
Luckily we can tell fsspec to cache the files locally before opening, this is denoted by
prepending the names as follows.

See discussion [here](https://discourse.pangeo.io/t/reading-goes-r-s3-netcdfs-from-an-aws-ec2-instance-is-it-possible-to-get-faster-speeds-than-from-my-local-machine/2440/13)
for more info.

In [5]:
def cached_path(date: datetime, forecast_hours: list, file_prefixes: list):
    return [f"simplecache::{u}" for u in ocean_path(date, forecast_hours, file_prefixes)]

In [6]:
cycle = datetime(1994,1,1,0)
cached_path(cycle, [0], ["ocn_"])

['simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/ocn_1994_01_01_00.nc']

## Use MOM6Dataset

Some of that is wrapped up under the hood in `MOM6Dataset`, we just need to give two inputs:
1. The filename mapping as defined above
2. A configuration yaml file

In [7]:
reader = MOM6Dataset(path_in=cached_path, config_filename="../scripts/config-replay.yaml")

We'll use this reader to open a dataset, note that the `fsspec_kwargs` is not necessary for local file reads, but necessary for reading from buckets like s3

In [8]:
%%time
ds = reader.open_dataset(cycle, fsspec_kwargs={"s3":{"anon":True}}, engine="h5netcdf")
ds

CPU times: user 2.56 s, sys: 1.2 s, total: 3.76 s
Wall time: 11.2 s


/Users/tsmith/miniconda3/envs/ufs2arco/lib/python3.11/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "z_l" starting at index 5. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/Users/tsmith/work/UFS2ARCO/src/UFS2ARCO/mom6.py:36: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  xds["time"] = xr.DataArray(
/Users/tsmith/work/UFS2ARCO/src/UFS2ARCO/mom6.py:45: UserWarning: Converting non-nanosecond precision timedelta values to nanosecond precision. This behavior can eventual

<xarray.Dataset>
Dimensions:     (xq: 360, yh: 320, z_l: 75, z_i: 76, time: 1, nv: 2, xh: 360,
                 yq: 320, zl: 75)
Coordinates:
  * xq          (xq) float64 -299.0 -298.0 -297.0 -296.0 ... 57.0 58.0 59.0 60.0
  * yh          (yh) float64 -77.77 -77.56 -77.34 -77.12 ... 89.48 89.69 89.9
  * z_l         (z_l) float64 0.5154 1.571 2.687 ... 5.698e+03 5.902e+03
  * z_i         (z_i) float64 0.0 1.031 2.112 ... 5.596e+03 5.8e+03 6.004e+03
    cftime      (time) object 1994-01-01 00:00:00
  * nv          (nv) float64 1.0 2.0
  * xh          (xh) float64 -299.5 -298.5 -297.5 -296.5 ... 56.5 57.5 58.5 59.5
  * yq          (yq) float64 -77.67 -77.45 -77.23 -77.01 ... 89.59 89.8 90.0
  * zl          (zl) float64 1.0 3.0 5.0 7.0 ... 3.338e+03 3.591e+03 5.111e+03
  * time        (time) datetime64[ns] 1994-01-01
    ftime       (time) timedelta64[ns] 00:00:00
Data variables: (12/23)
    uo          (time, z_l, yh, xq) float32 dask.array<chunksize=(1, 5, 320, 360), meta=np.ndarray>
    vo          (time, z_l, yq, xh) float32 dask.array<chunksize=(1, 5, 320, 360), meta=np.ndarray>
    so          (time, z_l, yh, xh) float32 dask.array<chunksize=(1, 5, 320, 360), meta=np.ndarray>
    temp        (time, z_l, yh, xh) float32 dask.array<chunksize=(1, 5, 320, 360), meta=np.ndarray>
    ho          (time, zl, yh, xh) float32 dask.array<chunksize=(1, 9, 320, 360), meta=np.ndarray>
    pbo         (time, yh, xh) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    ...          ...
    LwLatSens   (time, yh, xh) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    Heat_PmE    (time, yh, xh) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    time_bnds   (time, nv) object dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes:
    NumFilesInSet:  1
    title:          UFS
    grid_type:      regular
    grid_tile:      N/A

xarray gives a nice view of the data, we can look at a single variable to see how the data are laid out

In [9]:
ds["temp"]

<xarray.DataArray 'temp' (time: 1, z_l: 75, yh: 320, xh: 360)>
dask.array<open_dataset-temp, shape=(1, 75, 320, 360), dtype=float32, chunksize=(1, 5, 320, 360), chunktype=numpy.ndarray>
Coordinates:
  * yh       (yh) float64 -77.77 -77.56 -77.34 -77.12 ... 89.26 89.48 89.69 89.9
  * z_l      (z_l) float64 0.5154 1.571 2.687 ... 5.495e+03 5.698e+03 5.902e+03
    cftime   (time) object 1994-01-01 00:00:00
  * xh       (xh) float64 -299.5 -298.5 -297.5 -296.5 ... 56.5 57.5 58.5 59.5
  * time     (time) datetime64[ns] 1994-01-01
    ftime    (time) timedelta64[ns] 00:00:00
Attributes:
    units:          degC
    long_name:      Potential Temperature
    cell_methods:   area:mean z_l:mean yh:mean xh:mean time: mean
    time_avg_info:  average_T1,average_T2,average_DT

## Storing the dataset

The dataset has more variables than we wanted, but these are trimmed when we call `reader.store_dataset`.

Any arguments beyond the dataset are passed to `xarray.to_zarr`.
This is useful for appending multiple datasets in time (see e.g., ../scripts/read_from_s3.py)

By default, dask (which we're accessing through xarray under the hood) stores data to zarr using the chunksizes that are being used currently by dask (i.e., how the data are viewed in memory, currently).
However, it's more efficient to make zarr stores with smaller chunks, which we can then inflate later.
These smaller chunks are specified in our configuration yaml via chunks_out, and are used in the zarr store.

In [10]:
reader.chunks_out

{'time': 1, 'z_l': -1, 'z_i': -1, 'yh': 60, 'xh': 60, 'yq': 60, 'xq': 60}

In [11]:
reader.store_dataset(ds, mode="w")

Stored dataset at replay-1deg/forecast/mom6.zarr


In [12]:
xds = xr.open_zarr("replay-1deg/forecast/mom6.zarr/")

In [13]:
xds

<xarray.Dataset>
Dimensions:  (time: 1, yh: 320, xh: 360, z_l: 75, xq: 360, yq: 320)
Coordinates:
    cftime   (time) object dask.array<chunksize=(1,), meta=np.ndarray>
    ftime    (time) timedelta64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
  * time     (time) datetime64[ns] 1994-01-01
  * xh       (xh) float64 -299.5 -298.5 -297.5 -296.5 ... 56.5 57.5 58.5 59.5
  * xq       (xq) float64 -299.0 -298.0 -297.0 -296.0 ... 57.0 58.0 59.0 60.0
  * yh       (yh) float64 -77.77 -77.56 -77.34 -77.12 ... 89.26 89.48 89.69 89.9
  * yq       (yq) float64 -77.67 -77.45 -77.23 -77.01 ... 89.37 89.59 89.8 90.0
  * z_l      (z_l) float64 0.5154 1.571 2.687 ... 5.495e+03 5.698e+03 5.902e+03
Data variables:
    SSH      (time, yh, xh) float32 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
    so       (time, z_l, yh, xh) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
    temp     (time, z_l, yh, xh) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
    uo       (time, z_l, yh, xq) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
    vo       (time, z_l, xh, yq) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
Attributes:
    NumFilesInSet:  1
    grid_tile:      N/A
    grid_type:      regular
    title:          UFS

We can see that this only has the data variables we asked for in the config yaml file.

In [14]:
xds.temp

<xarray.DataArray 'temp' (time: 1, z_l: 75, yh: 320, xh: 360)>
dask.array<open_dataset-temp, shape=(1, 75, 320, 360), dtype=float32, chunksize=(1, 75, 60, 60), chunktype=numpy.ndarray>
Coordinates:
    cftime   (time) object dask.array<chunksize=(1,), meta=np.ndarray>
    ftime    (time) timedelta64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
  * time     (time) datetime64[ns] 1994-01-01
  * xh       (xh) float64 -299.5 -298.5 -297.5 -296.5 ... 56.5 57.5 58.5 59.5
  * yh       (yh) float64 -77.77 -77.56 -77.34 -77.12 ... 89.26 89.48 89.69 89.9
  * z_l      (z_l) float64 0.5154 1.571 2.687 ... 5.495e+03 5.698e+03 5.902e+03
Attributes:
    cell_methods:   area:mean z_l:mean yh:mean xh:mean time: mean
    long_name:      Potential Temperature
    time_avg_info:  average_T1,average_T2,average_DT
    units:          degC

And we can see this has a different chunking scheme than the original dataset, it has what we asked for in the config yaml.

In [15]:
reader.chunks_out

{'time': 1, 'z_l': -1, 'z_i': -1, 'yh': 60, 'xh': 60, 'yq': 60, 'xq': 60}